<a href="https://colab.research.google.com/github/tatianaesc/livroescd/blob/main/livro_ESCD_feature_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Livro Engenharia de Software para Ciência de Dados (Ed. Casa do Código)
Marcos Kalinowski, Tatiana Escovedo, Hugo Villamizar e Hélio Lopes

### Exemplo Prático de Feature Selection em Python

Nesta prática, estudar o processo de seleção de atributos, também conhecido como *feature selection*. Para tal, vamos trabalhar com o popular dataset Wine (disponível em https://archive.ics.uci.edu/ml/datasets/Wine), extraído a partir de uma análise química de vinhos cultivados na mesma região da Itália, mas derivados de três produtores diferentes. O objetivo deste dataset é identificar o produtor do vinho com base em 13 características químicas do vinho, ou seja, é um problema de Classificação. O dataset contém 178 instâncias (linhas), sendo 59 do produtor 1, 71 do do produtor 2 e 48 do produtor 3.

Todo o código está comentado, para facilitar o entendimento. Iniciaremos esta prática importando os pacotes necessários para o notebook:

In [ ]:
# Imports
import pandas as pd
import numpy as np
from numpy import set_printoptions
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier

# Novos imports
from sklearn.feature_selection import SelectKBest # para a Seleção Univariada
from sklearn.feature_selection import f_classif # para o teste ANOVA da Seleção Univariada
from sklearn.feature_selection import RFE # para a Eliminação Recursiva de Atributos

Vamos então carregar o dataset, converter seus dados para um dataframe, adicionar a coluna target e exibir as primeiras linhas para checarmos se tudo foi carregado com sucesso.

In [ ]:
# Carrega arquivo csv usando Pandas usando uma URL

# Informa a URL de importação do dataset
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"

# Informa o cabeçalho das colunas
colunas = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']

# Lê o arquivo utilizando as colunas informadas
dataset = pd.read_csv(url, names=colunas, skiprows=0, delimiter=',')

Em seguida, iremos preparar nossos dados para a aplicação dos métodos de seleção de atributos.

In [ ]:
# separando os atributos e a classe do dataset
array = dataset.values
X = array[:,0:8]
y = array[:,8]

# A semente (seed) pode ser qualquer número, e garante que os resultados possam ser reproduzidos de forma idêntica toda vez que o script for rodado. 
# Isto é muito importante quando trabalhamos com modelos ou métodos que utilizam de algum tipo de aleatoriedade.
seed = 7

### Seleção Univariada
https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html

A função **SelectKBest()** pode ser usada com diversos testes estatísticos para selecionar os atributos. Vamos usar o teste *ANOVA F-value* e selecionar os 4 melhores atributos que podem ser usados como variáveis preditoras.

Neste contexto, o teste *ANOVA F-value* estima o quanto cada característica de X é dependente da classe y, e é um teste especialmente adequado quanto temos a variável de saída categórica e as variáveis de entrada numéricas, como neste dataset.

Para saber mais: https://www.statisticshowto.com/probability-and-statistics/f-statistic-value-test/

In [ ]:
# Função para seleção de atributos
best_var = SelectKBest(score_func=f_classif, k=4)

# Executa a função de pontuação em (X, y) e obtém os atributos selecionados
fit = best_var.fit(X, y)

# Reduz X para os atributos selecionados
features = fit.transform(X)

# Resultados
print('\nNúmero original de atributos:', X.shape[1])
print('\nNúmero reduzido de atributos:', features.shape[1])

# Exibe os atributos orginais
print("\nAtributos Originais:", dataset.columns[0:8])

# Exibe as pontuações de cada atributos e os 4 escolhidas (com as pontuações mais altas): preg, plas, mass e age.
set_printoptions(precision=3) # 3 casas decimais
print("\nScores dos Atributos Originais:", fit.scores_)
print("\nAtributos Selecionados:", best_var.get_feature_names_out(input_features=dataset.columns[0:8]))

# Imprime o dataset apenas com as colunas selecionadas
print("\n", features)


Número original de atributos: 8

Número reduzido de atributos: 4

Atributos Originais: Index(['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age'], dtype='object')

Scores dos Atributos Originais: [ 39.67  213.162   3.257   4.304  13.281  71.772  23.871  46.141]

Atributos Selecionados: ['preg' 'plas' 'mass' 'age']

 [[  6.  148.   33.6  50. ]
 [  1.   85.   26.6  31. ]
 [  8.  183.   23.3  32. ]
 ...
 [  5.  121.   26.2  30. ]
 [  1.  126.   30.1  47. ]
 [  1.   93.   30.4  23. ]]


### Eliminação Recursiva de Atributos
https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html

Iremos aplicar a técnica de eliminação recursiva de atributos com um algoritmo de Regressão Logística (poderia ser qualquer classificador) para selecionar as 4 melhores variáveis preditoras.

In [ ]:
# Criação do modelo
modelo = LogisticRegression(max_iter=200)

# Eliminação Recursiva de Variáveis
rfe = RFE(modelo, n_features_to_select=4)
fit = rfe.fit(X, y)

# Print dos resultados
print("Atributos Originais:", dataset.columns[0:8])

# Exibe os atributos selecionados (marcados como True em "Atributos Selecionados" 
# e com valor 1 em "Ranking dos Atributos"): preg, plas, mass e pedi.
# (Basta mapear manualmente o índice dos nomes dos respectivos atributos)
print("\nAtributos Selecionados: %s" % fit.support_)
print("\nRanking de atributos: %s" % fit.ranking_)
print("\nQtd de melhores Atributos: %d" % fit.n_features_)
print("\nNomes dos Atributos Selecionados: %s" % fit.get_feature_names_out(input_features=dataset.columns[0:8]))

Atributos Originais: Index(['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age'], dtype='object')

Atributos Selecionados: [ True  True False False False  True  True False]

Ranking de atributos: [1 1 3 5 4 1 1 2]

Qtd de melhores Atributos: 4

Nomes dos Atributos Selecionados: ['preg' 'plas' 'mass' 'pedi']


### Importância de Atributos com ExtraTrees
http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html

Iremos construir um classificador ExtraTreesClassifier para calcular a importância dos atributos e posteriormente, selecionar quais iremos utilizar.

In [ ]:
# Criação do modelo para seleção de atributos
modelo = ExtraTreesClassifier(n_estimators=100)
modelo.fit(X,y)

# Exibe os atributos orginais
print("\nAtributos Originais:", dataset.columns[0:8])

# Exibe a pontuação de importância para cada atributo (quanto maior a pontuação, mais importante é o atributo). 
# Atributos selecionados: plas, mass, pedi, age.
print(modelo.feature_importances_)


Atributos Originais: Index(['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age'], dtype='object')
[0.109 0.231 0.1   0.081 0.073 0.14  0.122 0.144]


### Resumo dos atributos selecionados:

*   Técnica 1: preg, plas, mass e age
*   Técnica 2: preg, plas, mass e pedi
*   Técnica 3: plas, mass, pedi, age

OBS: É possível utilizar técnicas de *feature selection* como parte de um pipeline. Veja um exemplo no trecho de código a seguir:

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.linear_model import LogisticRegression 
from sklearn.pipeline import Pipeline 

pipeline = Pipeline([
  ('feature_selection', SelectKBest(score_func=f_classif, k=4)),
  ('classification', RandomForestClassifier(n_estimators=100, max_features=3, random_state=7))
])

Usando técnicas de *feature selection*, podemos decidir quais queremos selecionar para que sejam utilizados no treinamento do modelo de Machine Learning, o que seria o passo seguinte.

Nesta prática, vimos como utilizar três diferentes técnicas de feature selection. Note que, dependendo da técnica utilizada, os atributos selecionados poderão ser diferentes.